In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import display
from pathlib import Path

base = '../datasets/RepCount/'

In [92]:
base = '../datasets/RepCount'
train_csv = os.path.join(base, 'annotation/train.csv')
test_csv = os.path.join(base, 'annotation/test.csv')
val_csv = os.path.join(base, 'annotation/val.csv')

train_df = pd.read_csv(train_csv, index_col=0)
test_df = pd.read_csv(test_csv, index_col=0)
val_df = pd.read_csv(val_csv, index_col=0)

print(len(train_df), len(test_df), len(val_df))
print('total:', len(train_df) + len(test_df) + len(val_df))

train_df.head()

758 152 131
total: 1041


,type,name,count,L1,L2,L3,L4,L5,L6,L7,...,L293,L294,L295,L296,L297,L298,L299,L300,L301,L302
0,frontraise,train951.mp4,4.0,6.0,72.0,72.0,132.0,132.0,204.0,204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,frontraise,train952.mp4,10.0,13.0,62.0,62.0,103.0,103.0,126.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pullups,test1463.mp4,7.0,21.0,60.0,60.0,98.0,98.0,132.0,132.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,squant,test2340.mp4,4.0,0.0,86.0,86.0,155.0,155.0,218.0,219.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,front_raise,stu5_11.mp4,6.0,104.0,179.0,179.0,255.0,255.0,330.0,426.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Original YouTube data

In [93]:
original_data = Path(base, 'original_data/filename_mapping.xlsx')
ori = pd.read_excel(original_data, index_col=0)
print(len(ori))
ori[ori['vid']=='REjznoKN8Q8'].head(10)

800


,class,vid_st,vid,stuid,rename
693,squat,REjznoKN8Q8_143.mp4,REjznoKN8Q8,8,stu8_69
694,squat,REjznoKN8Q8_188.mp4,REjznoKN8Q8,3,stu3_67
779,squat,REjznoKN8Q8_188.mp4,REjznoKN8Q8,8,stu8_75


In [94]:
def min2sec(t):
    m, s = t.split(':')
    return int(m)*60 + int(s)

d_ = dict()
download_link_dir = Path(base, 'original_data/DownloadLink')
for fname in os.listdir(download_link_dir):
    with open(Path(download_link_dir, fname), 'r') as f:
        lines = f.readlines()
        for line in lines:
            vid, st, et = line.strip().split()
            st = min2sec(st)
            et = min2sec(et)
            vid_st = f'{vid}_{st}.mp4'
            d_[vid_st] = et

for row in ori.itertuples():
    vid_st = row.vid_st
    if vid_st in d_:
        ori.at[row.Index, 'et'] = int(d_[vid_st])
        ori.at[row.Index, 'st'] = int(vid_st[12:-4])

In [95]:
# add column 'name' to ori
ori['name'] = ori['rename'].apply(lambda x: x+'.mp4')
ori.drop(columns=['class'], inplace=True)
ori[ori['et'].notna()].head()


,vid_st,vid,stuid,rename,et,st,name
0,1pzpaYrhdOo_13.mp4,1pzpaYrhdOo,1,stu1_0,22.0,13.0,stu1_0.mp4
1,1pzpaYrhdOo_93.mp4,1pzpaYrhdOo,4,stu4_0,105.0,93.0,stu4_0.mp4
2,7wDx6mZDxA8_0.mp4,7wDx6mZDxA8,3,stu3_0,12.0,0.0,stu3_0.mp4
3,DcRj7jtKhk4_52.mp4,DcRj7jtKhk4,10,stu10_0,62.0,52.0,stu10_0.mp4
4,FyGBSs1X1qM_105.mp4,FyGBSs1X1qM,3,stu3_1,114.0,105.0,stu3_1.mp4


In [96]:
# reps to one column

train_df['reps'] = train_df[train_df.columns[3:]].values.tolist()
train_df.drop(train_df.columns[3:-1], axis=1, inplace=True)
train_df

,type,name,count,reps
0,frontraise,train951.mp4,4.0,"[6.0, 72.0, 72.0, 132.0, 132.0, 204.0, 204.0, ..."
1,frontraise,train952.mp4,10.0,"[13.0, 62.0, 62.0, 103.0, 103.0, 126.0, 126.0,..."
2,pullups,test1463.mp4,7.0,"[21.0, 60.0, 60.0, 98.0, 98.0, 132.0, 132.0, 1..."
3,squant,test2340.mp4,4.0,"[0.0, 86.0, 86.0, 155.0, 155.0, 218.0, 219.0, ..."
4,front_raise,stu5_11.mp4,6.0,"[104.0, 179.0, 179.0, 255.0, 255.0, 330.0, 426..."
...,...,...,...,...
753,others,stu13_5.mp4,5.0,"[5.0, 45.0, 45.0, 84.0, 84.0, 125.0, 125.0, 16..."
754,others,stu13_6.mp4,6.0,"[5.0, 49.0, 50.0, 89.0, 90.0, 133.0, 133.0, 17..."
755,pommelhorse,stu13_7.mp4,30.0,"[17.0, 44.0, 44.0, 69.0, 69.0, 96.0, 96.0, 121..."
756,pommelhorse,stu13_8.mp4,16.0,"[0.0, 24.0, 24.0, 47.0, 47.0, 68.0, 69.0, 91.0..."


In [97]:
# add YouTube ID and start time to annotation
def func(name):
    name = name.split('.')[0]
    row = ori[ori['rename'] == name]
    if len(row) == 0 or any(row['vid'].isna()):
        return None
    vid_st = row['vid_st'].values[0]

    vid = vid_st[:11]
    st = vid_st[12:-4]
    return vid, int(st)

train_df = train_df.merge(ori, how='left', on='name', suffixes=[None, None])
val_df = val_df.merge(ori, how='left', on='name', suffixes=[None, None])
test_df = test_df.merge(ori, how='left', on='name', suffixes=[None, None])

In [98]:
print(len(train_df[train_df['vid'].notna()]), len(val_df[val_df['vid'].notna()]), len(test_df[test_df['vid'].notna()]))


509 91 109


## Fixing the data
Many different names for the same thing.
Fixing it.

In [100]:
pushups = train_df[train_df['type'] == 'pushups']
pullups = train_df[train_df['type'] == 'pullups']
push_up = train_df[train_df['type'] == 'push_up']
pull_up = train_df[train_df['type'] == 'pull_up']

# replace column pushups to push_up, pullups to pull_up, squant to squat, jumpjacks to jump_jack
train_df['type'] = train_df['type'].replace(
    ['pushups', 'pullups', 'squant', 'jumpjacks', 'benchpressing', 'frontraise'],
    ['push_up', 'pull_up', 'squat', 'jump_jack', 'bench_pressing', 'front_raise'])

# val and test too
val_df['type'] = val_df['type'].replace(
    ['pushups', 'pullups', 'squant', 'jumpjacks', 'benchpressing', 'frontraise'],
    ['push_up', 'pull_up', 'squat', 'jump_jack', 'bench_pressing', 'front_raise'])

test_df['type'] = test_df['type'].replace(
    ['pushups', 'pullups', 'squant', 'jumpjacks', 'benchpressing', 'frontraise'],
    ['push_up', 'pull_up', 'squat', 'jump_jack', 'bench_pressing', 'front_raise'])


In [101]:
classes = train_df['type'].unique()
df1 = train_df.groupby('type').count()
df1 = df1[['count']].rename(columns={'count': 'count_train'})

df2 = val_df.groupby('type').count()
df2 = df2[['count']].rename(columns={'count': 'count_val'})

df3 = test_df.groupby('type').count()
df3 = df3[['count']].rename(columns={'count': 'count_test'})

# show df1 df2 df3 in one table
df = pd.concat([df1, df2, df3], axis=1)
df.index.name = 'type'
df.fillna(0, inplace=True)
df = df.astype(int)
df = df.reset_index()
df.sort_values('count_train', ascending=False)

,type,count_train,count_val,count_test
9,squat,101,16,18
6,pull_up,94,14,19
1,bench_pressing,93,13,19
2,front_raise,93,19,18
8,situp,93,18,20
7,push_up,88,18,16
3,jump_jack,76,15,26
5,pommelhorse,69,15,15
4,others,37,0,1
0,battle_rope,13,3,0


### Save all to new csv

Remove ['battle_rope', 'others', 'pommelhorse']

In [104]:
train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'
all_df = pd.concat([train_df, val_df, test_df], axis=0)

# drop types battle_rope, others, pommelhorse
to_drop = all_df[all_df['type'].isin(['battle_rope', 'others', 'pommelhorse'])].index
all_df.drop(to_drop, inplace=True)
all_df.rename(columns={'type': 'class', 'st': 'start', 'et': 'end'}, inplace=True)
# reorder columns
keep_cols = ['class', 'split', 'name', 'vid', 'start', 'end', 'count', 'reps']

all_df = all_df[keep_cols]
all_df

,class,split,name,vid,start,end,count,reps
0,front_raise,train,train951.mp4,NaN,NaN,NaN,4.0,"[6.0, 72.0, 72.0, 132.0, 132.0, 204.0, 204.0, ..."
1,front_raise,train,train952.mp4,NaN,NaN,NaN,10.0,"[13.0, 62.0, 62.0, 103.0, 103.0, 126.0, 126.0,..."
2,pull_up,train,test1463.mp4,NaN,NaN,NaN,7.0,"[21.0, 60.0, 60.0, 98.0, 98.0, 132.0, 132.0, 1..."
3,squat,train,test2340.mp4,NaN,NaN,NaN,4.0,"[0.0, 86.0, 86.0, 155.0, 155.0, 218.0, 219.0, ..."
4,front_raise,train,stu5_11.mp4,0_b43RbenY8,34.0,70.0,6.0,"[104.0, 179.0, 179.0, 255.0, 255.0, 330.0, 426..."
...,...,...,...,...,...,...,...,...
132,situp,test,stu8_57.mp4,zoL7eingzkQ,40.0,53.0,7.0,NaN
133,push_up,test,stu8_46.mp4,wS6Kp3k-zV8,70.0,112.0,19.0,NaN
134,push_up,test,stu4_47.mp4,t0QZmiXBC-Q,62.0,120.0,37.0,NaN
135,bench_pressing,test,stu2_6.mp4,QxLPG9FwAUU,90.0,123.0,4.0,NaN


In [105]:
all_df.to_csv(Path(base, 'all_data.csv'))